# Designing a synthesis route and executing it on a robot
In this example we show how to use the endpoints of the `rxn4chemistry` package. 
- We start from a target molecule in SMILES format.
- We then use the models to predict a retrosynthetic path.
- Once we have chosen the desired path, we start a new synthesis on the robotic hardware.
- We monitor the progress of the execution and finally download a pdf report from the analysis.

## Setup the logger

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s : %(message)s')

## Instantiating the wrapper
Setup the wrapper using a valida API key. You can get one on the IBM RXN website from [here](https://rxn.res.ibm.com/rxn/user/profile).

In [ ]:
from rxn4chemistry import RXN4ChemistryWrapper
api_key = 'API_KEY'
rxn4chemistry_wrapper = RXN4ChemistryWrapper(api_key=api_key)

## Create a project

Create a project, you can easily check the identifier associated to it in the response.

In [ ]:
response = rxn4chemistry_wrapper.create_project('rxn4chemistry_examples')

You can get your project id from the wrapper

In [ ]:
rxn4chemistry_wrapper.project_id

## Run a retrosynthesis prediction using the wrapper
Running a retrosynthesis is as easy as picking a molecule and calling a one-liner.

In this example, we will be predicting a path for [4-Bromoacetanilide](https://www.chemsynthesis.com/base/chemical-structure-2.html).

In [ ]:
smiles = 'CC(=O)NC1=CC=C(Br)C=C1'
response = rxn4chemistry_wrapper.predict_automatic_retrosynthesis(product=smiles, ai_model='2019-09-12')

### Check status and get the results
Rerun the following command until the status is 'SUCCESS', keep in mind the server allows only 5 requests per minute
and a timeout between consecutive requests of 2 seconds

In [ ]:
results = rxn4chemistry_wrapper.get_predict_automatic_retrosynthesis_results(response['prediction_id'])
results['status']

Upon 'SUCCESS' we can choose one of the returned retrosynthetic paths. The paths are sorted based on the scoring mechanism of the models.

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from IPython.display import display


def collect_reactions(tree):
    reactions = []
    if 'children' in tree and len(tree['children']):
        reactions.append(
            AllChem.ReactionFromSmarts('{}>>{}'.format(
                '.'.join([node['smiles'] for node in tree['children']]),
                tree['smiles']
            ), useSmiles=True)
        )
    for node in tree['children']:
        reactions.extend(collect_reactions(node))
    return reactions

In [ ]:
for index, path in enumerate(results['retrosynthetic_paths']):
    print('Showing path {} with confidence {}:'.format(index, path['confidence']))
    for reaction in collect_reactions(path):
        display(Chem.Draw.ReactionToImage(reaction))

**Hint**: You can visit your [rxn projects website](https://rxn.res.ibm.com/rxn/projects/) -> **rxn4chemistry_examples** -> **Attempts** -> **Retrosynthesis**. You should be able to see your new retrosynthesis prediction in the list!

## Starting the process on the robot
Once we have chosen the synthetic path, we can start the process on the robot
### Create a new synthesis object
Each retrosynthetic path predicted has a unique `sequenceId` that can be used to create a new synthesis

In [ ]:
path = results['retrosynthetic_paths'][0] # we choose the path with the highest score
response = rxn4chemistry_wrapper.create_synthesis_from_sequence(sequence_id=path['sequenceId'])
new_synthesis_id = response['synthesis_id']

### Inspect the synthesis plan
Once we create a synthesis and obtain a synthesis id, we can inspect the predicted actions required to perform the synthesis.

In [ ]:
import json
# get the entire list of actions for the entire synthesis, as well as a tree representation
synthesis_tree, reactions, actions = rxn4chemistry_wrapper.get_synthesis_plan(synthesis_id=new_synthesis_id)

# print the entire tree
print(json.dumps(synthesis_tree, indent=2))

# print the SMILES for each individual reaction (node in the tree)
for reaction in reactions:
    print('id: {} SMILES: {}'.format(reaction['id'], reaction['smiles']))

# print all the actions in the entire tree, in the order that they will be executed
for i, action in enumerate(actions, start=1):
    print('{}: {}'.format(i, action))

**Hint**: You can visit your [rxn projects website](https://rxn.res.ibm.com/rxn/projects/) -> **rxn4chemistry_examples** -> **Attempts** -> **Retrosynthesis**. Click **View** on your new retrosynthesis and in the new window click the **View in RoboRXN** button. You can now inspect your synthesis from the UI, modify the actions, as well as to specify additional actions, like `analysis`.

### Editing actions before submission
The actions predicted by the models can be edited before sending to the robot/simulator.
Actions editing takes place per tree node. We can use the `reactions` list returned in the previous step by `get_synthesis_plan()` to inspect/edit the actions of each node. In the following example we add an analysis action before the last step of the final reaction (the root of the synthesis tree).

In [ ]:
node_id = reactions[-1]['id']  # get the id of the root node
node_actions = rxn4chemistry_wrapper.get_node_actions(synthesis_id=new_synthesis_id, node_id=node_id)
print(json.dumps(node_actions, indent=2))

# add an analysis action before the last step
analysis_action = {'name': 'analysis', 'content': {'reactor_solute_concentration': {'value': 10}}}
node_actions.insert(-1, analysis_action)

# We also need to set a duration to the DrySolid action otherwise the validation will fail
node_actions[-3]['content']['duration'] = {'value': 60.0, 'quantity': None, 'unit': 'seconds'}

import time
time.sleep(10)  # avoid a "too frequent requests" error

# update the actions of this node
response = rxn4chemistry_wrapper.update_node_actions(synthesis_id=new_synthesis_id, node_id=node_id, actions=node_actions)

### Launch the synthesis
We can now start the synthesis process on the robotic hardware. Once the 'status' is 'DONE' the product is ready.

In [ ]:
synthesis_execution = rxn4chemistry_wrapper.start_synthesis(
    synthesis_id=new_synthesis_id
)
print(synthesis_execution['synthesis_execution_id'])

Once the synthesis has started, we can query the execution status.

In [ ]:
synthesis_status = rxn4chemistry_wrapper.get_synthesis_status(synthesis_id=synthesis_execution['synthesis_execution_id'])
print(synthesis_status['status'])

### Download any analysis reports that are available as pdf
If we have specified `analysis` actions, we can download the respective pdf reports once they are available.

In [ ]:
analysis_actions = rxn4chemistry_wrapper.get_synthesis_actions_with_spectrometer_pdf(synthesis_id=synthesis_execution['synthesis_execution_id'])
for action in analysis_actions:
    print(
        "synthesis_id: {}, reaction_id: {}, action_index: {}".format(
            action['synthesis_id'],
            action['node_id'],
            action['action_index']
        )
    )

time.sleep(10)  # avoid a "too frequent requests" error

pdfs = []
for action in analysis_actions:
    pdfs.append(
        rxn4chemistry_wrapper.get_synthesis_analysis_report_pdf(
            **action
        )
    )
print(pdfs[0])